In [ ]:
# Load DESeq2
if (!requireNamespace("DESeq2", quietly = TRUE)) {
  install.packages("BiocManager")
  BiocManager::install("DESeq2")
}
library(DESeq2)

# Define file paths
files <- c(
  "600" = "~/working/counts_STAR/600.tab",
  "601" = "~/working/counts_STAR/601.tab",
  "602" = "~/working/counts_STAR/602.tab"
)

# Read STAR ReadsPerGene.out.tab (column 2 = unstranded counts)
read_counts <- lapply(files, function(path) {
  dat <- read.table(path, header = FALSE)
  dat <- dat[, c(1, 2)]  # Column 1 = gene name, Column 2 = unstranded count
  colnames(dat) <- c("gene", "count")
  return(dat)
})

# Merge count data
merged <- Reduce(function(x, y) merge(x, y, by = "gene"), read_counts)
rownames(merged) <- merged$gene
merged <- merged[!grepl("^__", merged$gene), ]  # Remove STAR summary rows
count_matrix <- merged[, -1]
colnames(count_matrix) <- names(files)

# Sample metadata
col_data <- data.frame(
  row.names = colnames(count_matrix),
  condition = factor(c("control", "control", "treatment"))
)

# Create DESeq2 dataset
dds <- DESeqDataSetFromMatrix(countData = count_matrix, colData = col_data, design = ~condition)

# Run DESeq2
dds <- DESeq(dds)
res <- results(dds)
res <- na.omit(res)

# Save results
write.csv(as.data.frame(res), "deseq2_results.csv")

# MA Plot
pdf("MA_plot.pdf")
plotMA(res, ylim=c(-5,5))
dev.off()

# Volcano plot
pdf("volcano_plot.pdf")
with(as.data.frame(res), {
  plot(log2FoldChange, -log10(pvalue), pch=20, main="Volcano Plot", col=ifelse(padj < 0.05 & abs(log2FoldChange) > 1, "red", "gray"))
})
dev.off()
